# Database Reset and Test Data Insertion\n\nThis notebook resets the database and inserts test data for `internal_wiki` documents.\n\n> **Note:** This notebook uses the local PostgreSQL installation, not Docker.

In [4]:
# Import required modules
import sys
import os
from datetime import datetime

# Add the project root to the path if needed
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import directly from the module to avoid the __init__.py import chain
import iris.src.initial_setup.db_config as db_config

# Connect to the database
conn = db_config.connect_to_db("local")

Connecting to maven-finance at localhost:5432...
Connected successfully! 🎉


In [5]:
# Check current data
if conn:
    with conn.cursor() as cur:
        # Check catalog records
        cur.execute("SELECT COUNT(*) FROM apg_catalog")
        catalog_count = cur.fetchone()[0]
        print(f"Current apg_catalog record count: {catalog_count}")
        
        # Check content records
        cur.execute("SELECT COUNT(*) FROM apg_content")
        content_count = cur.fetchone()[0]
        print(f"Current apg_content record count: {content_count}")
        
        # Show summary of document sources
        cur.execute("SELECT document_source, COUNT(*) FROM apg_catalog GROUP BY document_source")
        sources = cur.fetchall()
        print("\nDocument sources in apg_catalog:")
        for source, count in sources:
            print(f"  - {source}: {count} records")

Current apg_catalog record count: 3
Current apg_content record count: 6

Document sources in apg_catalog:
  - internal_wiki: 3 records


In [6]:
# Clear all existing data
if conn:
    with conn.cursor() as cur:
        print("Clearing existing data...")
        cur.execute("TRUNCATE TABLE apg_catalog, apg_content RESTART IDENTITY CASCADE;")
        conn.commit()
        print("All data cleared.")

Clearing existing data...
All data cleared.


In [7]:
# Insert catalog records
if conn:
    with conn.cursor() as cur:
        print("Inserting catalog records...")
        
        catalog_records = [
            (
                'internal_wiki', 
                'ifrs_standard', 
                'IFRS_15_Revenue', 
                'IFRS 15 Revenue from Contracts with Customers: Addresses the principles for recognizing revenue from contracts with customers. Establishes a five-step model for revenue recognition.',
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                'IFRS_15_Revenue.md',
                '.md',
                '//wiki/finance/standards/',
            ),
            (
                'internal_wiki', 
                'ifrs_standard', 
                'IAS_12_IncomeTaxes', 
                'IAS 12 Income Taxes: Prescribes the accounting treatment for income taxes. Addresses the current and future tax consequences of transactions and events.',
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                'IAS_12_IncomeTaxes.md',
                '.md',
                '//wiki/finance/standards/',
            ),
            (
                'internal_wiki', 
                'internal_memo', 
                'Q1_2025_Financial_Analysis', 
                'Internal analysis of Q1 2025 financial performance across business units. Includes variance analysis and forecasting adjustments.',
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                datetime.fromisoformat('2025-03-27 13:52:12-04:00'),
                'Q1_2025_Financial_Analysis.md',
                '.md',
                '//wiki/finance/memos/',
            ),
        ]
        
        for record in catalog_records:
            cur.execute("""
                INSERT INTO apg_catalog 
                (document_source, document_type, document_name, document_description, 
                date_created, date_last_modified, file_name, file_type, file_path)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, record)
        
        conn.commit()
        print("Catalog records inserted.")

Inserting catalog records...
Catalog records inserted.


In [8]:
# Insert content records
if conn:
    with conn.cursor() as cur:
        print("Inserting content records...")
        
        content_records = [
            # IFRS 15 Records
            (
                'internal_wiki', 'ifrs_standard', 'IFRS_15_Revenue', 0, None, None,
                """# Record Information

- **Year:** 15
- **Published:** 2014

# Standards and Classification

- **IFRS Standard 15**: Revenue from Contracts with Customers

This standard establishes a comprehensive framework for determining when to recognize revenue and how much revenue to recognize. The core principle is that an entity should recognize revenue to depict the transfer of promised goods or services to customers in an amount that reflects the consideration to which the entity expects to be entitled in exchange for those goods or services."""
            ),
            (
                'internal_wiki', 'ifrs_standard', 'IFRS_15_Revenue', 1, 'Five-Step Model', 
                'The five-step model for revenue recognition under IFRS 15',
                """# Five-Step Model for Revenue Recognition

1. **Identify the contract(s) with a customer**
2. **Identify the performance obligations in the contract**
3. **Determine the transaction price**
4. **Allocate the transaction price to the performance obligations**
5. **Recognize revenue when (or as) the entity satisfies a performance obligation**

This model applies to all contracts with customers except leases, insurance contracts, financial instruments, and certain non-monetary exchanges."""
            ),
            
            # IAS 12 Records
            (
                'internal_wiki', 'ifrs_standard', 'IAS_12_IncomeTaxes', 0, None, None,
                """# Record Information

- **Year:** 12
- **Revised:** 2023

# Standards and Classification

- **IAS 12**: Income Taxes

This standard prescribes the accounting treatment for income taxes. It addresses both current tax and deferred tax consequences."""
            ),
            
            # Q1 2025 Financial Analysis Records
            (
                'internal_wiki', 'internal_memo', 'Q1_2025_Financial_Analysis', 0, None, None,
                """# Record Information

- **Quarter:** Q1
- **Year:** 2025
- **Department:** Finance

# Executive Summary

This document presents the financial analysis for Q1 2025. Key performance indicators show a 12% increase in revenue compared to the previous quarter, with operating margin improving by 2.5 percentage points."""
            ),
            (
                'internal_wiki', 'internal_memo', 'Q1_2025_Financial_Analysis', 1, 'Revenue Analysis', 
                'Breakdown of Q1 2025 revenue by business unit and product line',
                """# Revenue Analysis

## Business Unit Performance
- **North America**: $245.6M (+15.2% YoY)
- **EMEA**: $198.3M (+8.7% YoY)
- **APAC**: $156.9M (+22.3% YoY)

## Product Line Performance
- **Enterprise Solutions**: $312.4M (+18.1% YoY)
- **Consumer Products**: $189.5M (+5.3% YoY)
- **Professional Services**: $98.9M (+12.8% YoY)"""
            ),
            (
                'internal_wiki', 'internal_memo', 'Q1_2025_Financial_Analysis', 2, 'Cost Structure', 
                'Analysis of Q1 2025 cost structure and efficiency metrics',
                """# Cost Structure Analysis

## Operating Expenses
- **Cost of Goods Sold**: $285.3M (47.5% of revenue)
- **R&D**: $84.7M (14.1% of revenue)
- **Sales & Marketing**: $95.2M (15.8% of revenue)
- **G&A**: $42.6M (7.1% of revenue)

## Efficiency Metrics
- **Gross Margin**: 52.5% (+1.8pp YoY)
- **Operating Margin**: 15.5% (+2.5pp YoY)
- **Net Margin**: 12.3% (+1.9pp YoY)"""
            ),
        ]
        
        for record in content_records:
            cur.execute("""
                INSERT INTO apg_content
                (document_source, document_type, document_name, section_id, section_name, section_summary, section_content)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, record)
        
        conn.commit()
        print("Content records inserted.")

Inserting content records...
Content records inserted.


In [9]:
# Verify the inserted data
if conn:
    with conn.cursor() as cur:
        # Check catalog records
        cur.execute("SELECT COUNT(*) FROM apg_catalog")
        catalog_count = cur.fetchone()[0]
        print(f"Final apg_catalog record count: {catalog_count}")
        
        # Check content records
        cur.execute("SELECT COUNT(*) FROM apg_content")
        content_count = cur.fetchone()[0]
        print(f"Final apg_content record count: {content_count}")
        
        # Show catalog records
        print("\nApg_catalog records:")
        cur.execute("SELECT id, document_source, document_type, document_name FROM apg_catalog")
        for row in cur.fetchall():
            print(f"  - ID {row[0]}: {row[1]} / {row[2]} / {row[3]}")
            
        # Show content records summary
        print("\nApg_content records:")
        cur.execute("""
            SELECT c.document_name, count(ct.id)
            FROM apg_catalog c
            JOIN apg_content ct ON c.document_source = ct.document_source 
                AND c.document_type = ct.document_type 
                AND c.document_name = ct.document_name
            GROUP BY c.document_name
        """)
        for doc_name, section_count in cur.fetchall():
            print(f"  - {doc_name}: {section_count} sections")

Final apg_catalog record count: 3
Final apg_content record count: 6

Apg_catalog records:
  - ID 1: internal_wiki / ifrs_standard / IFRS_15_Revenue
  - ID 2: internal_wiki / ifrs_standard / IAS_12_IncomeTaxes
  - ID 3: internal_wiki / internal_memo / Q1_2025_Financial_Analysis

Apg_content records:
  - IAS_12_IncomeTaxes: 1 sections
  - IFRS_15_Revenue: 2 sections
  - Q1_2025_Financial_Analysis: 3 sections


In [10]:
# Close the connection when done
if conn:
    conn.close()
    print("Database connection closed.")

Database connection closed.
